In [ ]:
import pandas as pd
import re
from datetime import datetime
import numpy as np
import os
import tempfile

In [ ]:
# Read the parquet file containing listing details
df = pd.read_parquet("data/as24_listing_details.parquet")

In [ ]:
def clean_price(val):
    s = str(val)
    # Remove currency and spaces
    s = re.sub(r"[^\d,]", "", s)
    # If there are 4 digits after the comma, drop the last digit
    s = re.sub(r"(\d+),(\d{4})$", lambda m: f"{m.group(1)},{m.group(2)[:-1]}", s)
    # Remove any remaining non-digit/non-comma
    s = re.sub(r"[^\d,]", "", s)
    return s

df["price_num"] = pd.to_numeric(df["price"].apply(clean_price).str.replace(",", ""), errors='coerce')
df["Mileage_num"] = pd.to_numeric(df["Mileage"].astype(str).str.replace(r"[^\d.,]", "", regex=True).str.replace(",", ""), errors='coerce')

df = df[df["Mileage_num"] < 1000000]
df["Mileage_inv"] = df["Mileage_num"].apply(lambda x: 1/x if x > 0 else .999999)

df['log_price'] = np.log1p(df['price_num'])
df['log_km'] = np.log1p(df['Mileage_num'])


# Parse registration year/month
if "First registration" in df.columns:
    df["registration_year"] = df["First registration"].apply(lambda x: int(re.search(r"(\d{4})", str(x)).group()) if pd.notnull(x) and re.search(r"(\d{4})", str(x)) else None)
    df["registration_month"] = df["First registration"].apply(lambda x: int(re.search(r"(\d{2})/(\d{4})", str(x)).group(1)) if pd.notnull(x) and re.search(r"(\d{2})/(\d{4})", str(x)) else None)
    df["age"] = 2025 - df["registration_year"]

# Parse power (kW and hp)
if "Power" in df.columns:
    df["Power_kW"] = df["Power"].str.extract(r"(\d+) kW").astype(float)
    df["Power_hp"] = df["Power"].str.extract(r"(\d+) hp").astype(float)

# Seats, Doors, Engine size, Cylinders, Empty weight, CO2 emissions
for col in ["Seats", "Engine size", "Cylinders", "Empty weight", "CO₂-emissions"]:
    if col in df.columns:
        df[col + "_num"] = pd.to_numeric(df[col].astype(str).str.replace(",", "").str.replace(r"[^\d.,]", "", regex=True), errors='coerce')

# Flatten equipment dict to string list
if "equipment" in df.columns:
    def flatten_equipment(eq):
        if pd.isnull(eq): return ""
        if isinstance(eq, str):
            try:
                import ast
                eq = ast.literal_eval(eq)
            except Exception:
                return eq
        if isinstance(eq, dict):
            return ", ".join([item for sublist in eq.values() for item in (sublist if isinstance(sublist, list) else [sublist])])
        return str(eq)
    df["equipment_flat"] = df["equipment"].apply(flatten_equipment)

# Colour, Paint, Upholstery colour, Upholstery dummies
for col in ["Colour", "Paint"]:
    if col in df.columns:
        df[col + "_clean"] = df[col].astype(str).str.lower().str.strip()
        df = pd.get_dummies(df, columns=[col + "_clean"], prefix=col, drop_first=True)

# Dummies
#df = pd.get_dummies(df, columns=["seller_type"], prefix="SellerType", drop_first=True)

# Combine "Fuel type" and "Other fuel types" into a single column
def combine_fuel_type(row):
    fuel = str(row["Fuel type"]).strip()
    other = str(row["Other fuel types"]).strip()
    if fuel and fuel.lower() != "none":
        if other and other.lower() != "none":
            # If both are present and different, join them
            if fuel != other:
                return f"{fuel}/{other}"
        return fuel
    elif other and other.lower() != "none":
        return other
    else:
        return None

def categorize_fuel_type(fuel):
    fuel = str(fuel).lower()
    if 'electric' in fuel:
        return 'Electric/Hybrid'
    if any(x in fuel for x in ['diesel', 'biodiesel', 'vegetable oil']):
        return 'Diesel'
    if any(x in fuel for x in ['gasoline', 'super', 'regular', 'benzine']):
        return 'Petrol'
    if any(x in fuel for x in ['cng', 'lpg', 'domestic gas', 'biogas']):
        return 'Gas'
    if fuel in ['others', '-']:
        return 'Other/Unknown'
    return 'Other/Unknown'

df["Fuel_type_combined"] = df.apply(combine_fuel_type, axis=1)
df['FuelCategory'] = df['Fuel_type_combined'].apply(categorize_fuel_type)
fuelcat_dummies = pd.get_dummies(df['FuelCategory'], prefix='FuelCat', drop_first=False)
df = pd.concat([df, fuelcat_dummies], axis=1)


df["Gearbox_automatic"] = df["Gearbox"].astype(str).str.lower().str.contains("automatic").astype(int)
df["Smoker"] = df["Non-smoker vehicle"].astype(str).str.lower().str.contains("no|nein|non|false|0").astype(int)
df["PrevOwnerKnown"] = df["Previous owner"].notnull().astype(int)
def last_service_recent(x):
    if pd.isnull(x): return 0
    match = re.search(r"(\d{2})/(\d{4})", str(x))
    if match:
        month, year = int(match.group(1)), int(match.group(2))
        today = datetime.today()
        service_date = datetime(year, month, 1)
        delta = (today.year - service_date.year)*12 + (today.month - service_date.month)
        return int(delta <= 12)
    match2 = re.search(r"(\d{4})", str(x))
    if match2:
        year = int(match2.group(1))
        today = datetime.today()
        return int(today.year - year <= 1)
    return 0
df["LastServiceRecent"] = df["Last service"].apply(last_service_recent)
df["Drivetrain_4WD"] = df["Drivetrain"].astype(str).str.lower().str.contains("4wd|awd|quattro|allrad|four-wheel").astype(int)
df["Upholstery_leather"] = df["Upholstery"].astype(str).str.lower().str.contains("leather|leder").astype(int)
df["EmissionClass_clean"] = df["Emission class"].astype(str).str.extract(r"(euro \d+[a-zA-Z\-]*)", flags=re.IGNORECASE, expand=False).str.lower().fillna("")
df = pd.get_dummies(df, columns=["EmissionClass_clean"], prefix="EmissionClass", drop_first=True)
df['euro_score'] = df['Emission class'].map({
    'Euro 1': 1.0,
    'Euro 2': 2.0,
    'Euro 3': 3.0,
    'Euro 4': 4.0,
    'Euro 5': 5.0,
    'Euro 6': 6.0,
    'Euro 6b': 6.1,
    'Euro 6c': 6.15,
    'Euro 6d-TEMP': 6.2,
    'Euro 6d': 6.3,
    'Euro 6e': 6.4,
    None: np.nan
})
df["EmissionClass_compliant"] = (df["euro_score"]>= 6).astype(int)
df["Warranty_present"] = df["Warranty"].notnull().astype(int)
df["Warranty_months"] = df["Warranty"].astype(str).str.extract(r"(\d{1,2}) ?months", expand=False).astype(float)
df["FuelConsumption_l100km"] = df["Fuel consumption"].astype(str).str.extract(r"([\d.]+) ?l/100 ?km", expand=False).astype(float)
COMMERCIAL_BRANDS = [
    'Mercedes-Benz', 'Volkswagen', 'Ford', 'Opel', 'Renault', 'Peugeot', 'Iveco', 'Citroen', 'Fiat', 'MAN'
]
# Extract brand from model_name and create a Brand column
def extract_brand(model_name):
    for brand in COMMERCIAL_BRANDS:
        if str(model_name).startswith(brand):
            return brand
    return "Other"

df["Brand"] = df["model_name"].apply(extract_brand)

# Create dummies for each brand
df = pd.get_dummies(df, columns=["Brand"], prefix="Brand", drop_first=False)

# Map equipment features to binary columns and encode feature groups
equipment_features = {
    "sliding_door_right": "Sliding door right",
    "sliding_door_left": "Sliding door left",
    "trailer_hitch": "Trailer hitch",
    "roof_rack": "Roof rack",
    "cargo_barrier": "Cargo barrier",
    "double_cabin": "Double cabin",
    "tail_lift": "Tail-lift",
    "differential_lock": "Differential lock",
    "automatic_climate_control": "Automatic climate control",
    "seat_heating": "Seat heating",
    "armrest": "Armrest",
    "hill_holder": "Hill Holder",
    "start_stop_system": "Start-stop system",
    "winter_package": "Winter package",
    "auxiliary_heating": "Auxiliary heating",
    "lane_departure_warning": "Lane departure warning system",
    "blind_spot_monitor": "Blind spot monitor",
    "adaptive_cruise_control": "Adaptive Cruise Control",
    "traffic_sign_recognition": "Traffic sign recognition",
    "driver_drowsiness_detection": "Driver drowsiness detection",
    "emergency_brake_assistant": "Emergency brake assistant",
    "particle_filter": "Particle filter",
    "navigation_system": "Navigation system",
    "park_distance_control": "Park Distance Control",
    "parking_assist_camera": "Parking assist system camera",
    "led_headlights": "LED Headlights",
    "full_led_headlights": "Full-LED headlights"
}

feature_groups = {
    "operational_utility": [
        "sliding_door_right", "sliding_door_left", "trailer_hitch",
        "roof_rack", "cargo_barrier", "double_cabin",
        "tail_lift", "differential_lock"
    ],
    "driver_comfort": [
        "automatic_climate_control", "seat_heating", "armrest",
        "hill_holder", "start_stop_system", "winter_package",
        "auxiliary_heating"
    ],
    "safety_compliance": [
        "lane_departure_warning", "blind_spot_monitor", "adaptive_cruise_control",
        "traffic_sign_recognition", "driver_drowsiness_detection",
        "emergency_brake_assistant"
    ],
    "modern_tech": [
        "navigation_system", "park_distance_control", "parking_assist_camera",
        "led_headlights", "full_led_headlights"
    ]
}

for group, features in feature_groups.items():
    df[group] = df["equipment_flat"].apply(
        lambda x: int(any(eq_name in str(x) for eq_name in [equipment_features[f] for f in features]))
    )


# Join location from listings_bs4 on url
listings_bs4 = pd.read_parquet("data/as24_listings_bs4.parquet")
df = df.drop(columns=["location"], errors="ignore")  # Remove old if exists
df = df.merge(listings_bs4[["url", "location"]], on="url", how="left")
# Extract country code (e.g., 'NL') from location string
def extract_country_code(location):
    if pd.isnull(location):
        return None
    match = re.search(r"\b([A-Z]{2})-\d{4,}", str(location))
    if match:
        return match.group(1)
    return None

df["country_code"] = df["location"].apply(extract_country_code)
df = pd.get_dummies(df, columns=["country_code"], prefix="Country", drop_first=False)

# Convert boolean dummies to int
dummy_cols = [col for col in df.columns if df[col].dtype == bool]
df[dummy_cols] = df[dummy_cols].astype(int)

# Remove illegal characters from all string columns
def clean_utf8(val):
    if isinstance(val, str):
        val = re.sub(r"[\x00-\x08\x0b-\x0c\x0e-\x1f]", "", val)
        return val.encode("utf-8", "ignore").decode("utf-8", "ignore")
    return val

for colname in df.select_dtypes(include=["object"]).columns:
    df[colname] = df[colname].apply(clean_utf8)

# Save cleaned file
output_file = "data/as24ListingDetails_cleaned.csv"
output_parquet = "data/as24ListingDetails_cleaned.parquet"
df.to_parquet(output_parquet, index=False)
df.to_csv(output_file, index=False)
df.head()

,model_name,price,seller_type,Mileage,Gearbox,First registration,Fuel type,Power,Seller,Body type,...,modern_tech,location,Country_AT,Country_BE,Country_DE,Country_ES,Country_FR,Country_IT,Country_LU,Country_NL
0,Mercedes-Benz Vito114 cdi,"€ 19,900",,"30,000 km",Manual,09/2020,Diesel,100 kW (136 hp),Dealer,Transporter,...,0,Responsabile Vendite • IT-36040 Sarego- Vi,0,0,0,0,0,1,0,0
1,Mercedes-Benz CitanMercedes Citan 108CDi,"€ 18,9951",,"29,538 km",Manual,02/2023,Diesel,55 kW (75 hp),Dealer,Transporter,...,1,Bjorn Sneppe Sneppe • BE-8020 Oostkamp,0,1,0,0,0,0,0,0
2,"Mercedes-Benz Citan1.5 CDI A2 BE, Alu velgen, ...","€ 12,9701",,"50,407 km",Manual,04/2020,Diesel,59 kW (80 hp),Dealer,Transporter,...,1,BE-3740 Bilzen,0,1,0,0,0,0,0,0
3,Mercedes-Benz V 200AUTOMAAT - DOCA - NAVI - PS...,"€ 31,9971",,"171,314 km",Automatic,06/2017,Diesel,120 kW (163 hp),Dealer,Transporter,...,1,Maxime Debersaques • BE-9070 Destelbergen,0,1,0,0,0,0,0,0
4,Mercedes-Benz Vito119 cdi long auto my20,"€ 17,5001",,"226,426 km",Semi-automatic,04/2021,Diesel,140 kW (190 hp),Dealer,Transporter,...,0,Leonardo Colizzi • IT-36050 Bolzano Vicentino ...,0,0,0,0,0,1,0,0


In [6]:
# Show all calculated/cleaned columns in the dataframe
calculated_cols = [
    col for col in df.columns
    if (
        col.endswith('_num') or
        col.endswith('_flat') or
        col.endswith('_clean') or
        col in [
            'price_num', 'Mileage_num', 'registration_year', 'registration_month', 'age',
            'Power_kW', 'Power_hp', 'NonSmoker', 'PrevOwnerKnown', 'LastServiceRecent',
            'Drivetrain_4WD', 'Upholstery_leather', 'Warranty_present', 'Warranty_months',
            'FuelConsumption_l100km'
        ] or
        col.startswith('SellerType_') or
        col.startswith('GearboxType_') or
        col.startswith('FuelType_') or
        col.startswith('EmissionClass_')
    )
]

print(calculated_cols)

['price_num', 'Mileage_num', 'registration_year', 'registration_month', 'age', 'Power_kW', 'Power_hp', 'Seats_num', 'Engine size_num', 'Cylinders_num', 'Empty weight_num', 'CO₂-emissions_num', 'equipment_flat', 'PrevOwnerKnown', 'LastServiceRecent', 'Drivetrain_4WD', 'Upholstery_leather', 'EmissionClass_euro 1', 'EmissionClass_euro 2', 'EmissionClass_euro 3', 'EmissionClass_euro 4', 'EmissionClass_euro 5', 'EmissionClass_euro 6', 'EmissionClass_euro 6b', 'EmissionClass_euro 6c', 'EmissionClass_euro 6d', 'EmissionClass_euro 6d-temp', 'EmissionClass_euro 6e', 'EmissionClass_compliant', 'Warranty_present', 'Warranty_months', 'FuelConsumption_l100km']


In [7]:
from collections import Counter, defaultdict
import ast

# Aggregate equipment counts by category
equipment_counts = defaultdict(Counter)

def parse_equipment(eq_str):
    if pd.isnull(eq_str):
        return {}
    if isinstance(eq_str, dict):
        return eq_str
    if isinstance(eq_str, str):
        try:
            eq_dict = ast.literal_eval(eq_str)
            if isinstance(eq_dict, dict):
                return eq_dict
        except Exception:
            return {}
    return {}

for eq_str in df['equipment'].dropna():
    eq_dict = parse_equipment(eq_str)
    for category, items in eq_dict.items():
        if isinstance(items, list):
            equipment_counts[category].update(items)
        elif isinstance(items, str):
            equipment_counts[category].update([items])

# Print counts for each category
for category, counter in equipment_counts.items():
    print(f"{category}:")
    for item, count in counter.most_common():
        print(f"  {item}: {count}")

Comfort & Convenience:
  Air conditioning: 52847
  Power windows: 47972
  Electrical side mirrors: 40593
  Cruise control: 39148
  Park Distance Control: 38638
  Multi-function steering wheel: 32925
  Parking assist system sensors rear: 32600
  Sliding door right: 30284
  Armrest: 28941
  Start-stop system: 28261
  Hill Holder: 25935
  Rain sensor: 23160
  Light sensor: 22504
  Parking assist system camera: 22323
  Navigation system: 21954
  Parking assist system sensors front: 15809
  Lumbar support: 14126
  Automatic climate control: 13788
  Leather steering wheel: 13307
  Tinted windows: 10575
  Seat heating: 10350
  Electrically heated windshield: 7991
  Auxiliary heating: 4040
  Sliding door left: 3798
  Keyless central door lock: 3678
  Parking assist system self-steering: 2635
  Leather seats: 2532
  Fold flat passenger seat: 1565
  360° camera: 1555
  Automatic climate control, 2 zones: 1492
  Split rear seats: 1418
  Heated steering wheel: 1314
  Electrically adjustable seats: